In [1]:
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning import Trainer
from policy import ForwardPolicy, BackwardPolicy
from gflownet.gflownet import GFlowNet
from gflownet.dataset import MatrixDataModule

def main():
    # Initialize the data module
    matrix_dir = 'data/small_ILU'  # Replace with your actual directory
    data_module = MatrixDataModule(matrix_directory=matrix_dir, batch_size=1)
                                   
    node_features = -1
    input_dim = 1
    hidden_dim = 4
    max_num_actions = 500
    back_action_size = -1
    #Initialize the model
    forward_policy = ForwardPolicy(node_features=node_features, hidden_dim=hidden_dim, max_num_actions=max_num_actions)
    backward_policy = BackwardPolicy(input_dim=input_dim, hidden_dim=hidden_dim, max_num_actions=max_num_actions)

    logger = TensorBoardLogger("tb_logs", name="gflownet")

    model = GFlowNet(forward_policy=forward_policy, backward_policy=backward_policy, no_sampling_batch=2, lr=0.00002)

    # Initialize the PyTorch Lightning Trainer
    trainer = Trainer(max_epochs=3, logger=logger)  # You can modify trainer arguments as needed

    # Train the model
    trainer.fit(model, data_module)

if __name__ == '__main__':
    main()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/tonylizza/opt/anaconda3/envs/ML2/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:106: UserWarning: You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.
  rank_zero_warn("You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.")
/Users/tonylizza/opt/anaconda3/envs/ML2/lib/python3.9/site-packages/pytorch_lightning/core/optimizer.py:329: RuntimeWarning: The lr scheduler dict contains the key(s) ['monitor'], but the keys will be ignored. You need to call `lr_scheduler.step()` manually in manual optimization.
  rank_zero_warn(
/Users/tonylizza/opt/anaconda3/envs/ML2/lib/python3.9/site-packages/pytorch_lightning/utilities/model_summary/model_summary.py:411: UserWarning: A layer with UninitializedParameter was found. Thus, the total number of parameters

Training: 0it [00:00, ?it/s]

/Users/tonylizza/Documents/Machine_Learning/Thesis_Coding/gflownet-spai/gflownet/dataset.py:39: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/miniforge3/conda-bld/pytorch-recipe_1670284382755/work/torch/csrc/utils/tensor_new.cpp:233.)
  ilu_indices = torch.tensor([ilu_matrix.row, ilu_matrix.col], dtype=torch.long)


Loss: 126.84998321533203


/Users/tonylizza/opt/anaconda3/envs/ML2/lib/python3.9/site-packages/pytorch_lightning/utilities/data.py:85: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 20. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(
2024-10-02 18:08:26.010122: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


...

`Trainer.fit` stopped: `max_epochs=3` reached.
  0%|          | 0/1 [00:00<?, ?it/s]/Users/tonylizza/opt/anaconda3/envs/ML2/lib/python3.9/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:437: SparseEfficiencyWarning: spilu converted its input to CSC format
  warn('spilu converted its input to CSC format',


Type of b_vector: <class 'numpy.ndarray'>


  0%|          | 0/1 [00:07<?, ?it/s]

0.32812318921093814
0.15179589887931275
0.10120974737554654
0.09827242672107069
0.09660496166578185
0.09550885211837341
0.06930874396518417
0.012729815114667175
0.0068508800875989794
0.0027690650855343533
0.0012317148149761203
0.000316327188247506
0.0001989472706301368
9.043720845232405e-05
8.449511459155003e-05
3.129417134234208e-05
1.0424079916612562e-05
1.8747002397253028e-06
GMRES converged successfully.
GMRES no preconditioner
0.0026174187636077376
0.0021290864012168892
0.0019634946844222364
0.001879777410405055
0.0013779486768492582
0.0013719907206318712
0.001370507822265031
0.0013402971904138018
0.0011809522869048574
0.0007974278023325054
0.0006200569434583073
3.33762528525477e-05
7.626650324963723e-06
1.737371358856771e-06
1.7371487746582079e-06
1.0534158322314024e-06
1.0531935303641418e-06
1.0471187475693089e-06
8.559369745298563e-08
8.292919254468309e-08
8.292919256451806e-08
8.292919256346775e-08
8.292919256341593e-08
8.292919255848151e-08
8.292919254818495e-08
8.29291925407

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()